I need to schedule meetings with PhDs on the services I offer for them as PhDs. I created a master calendar of available time slots I would block out to present for them. I have 6 groups of PhDs, and I don't want to supply them with all of the dates and slots, I want to only provide them a selection of those dates. I don't want to go too far with randomly selecting, but I don't want to do the selection myself, so I wrote this to take the available events and output new calendars per group. 

First off, import the first thing I know I'll need, `random.choice`, and point to the calendar file for the master schedule. 

In [ ]:
from random import choice as rc

calendar = r"C:\Users\SPEED001\Documents\GitHub\random\PhDCalendar.ics"

I have the different groups in an excel table, but I don't really need to import the actual file, I can just split a string by newline. 

In [ ]:
phd_groups_txt = """Energy and Resources
Economic Geography, Geography Education, International Development Studies, and Spatial Planning
Environmental Governance
Envrionmental Sciences
Innovation Studies and Urban Futures Studio
Urban Geography"""
phd_groups = phd_groups_txt.split("\n")
print(phd_groups)

Read in the calendar file. 

In [ ]:
with open(calendar, "r") as fcal: 
    calendar_txt = fcal.read()

Get a preview of the file.

In [ ]:
print(calendar_txt[:200])

Split the calendar up into events. I don't want to  really change much, so I just split the calendar by `BEGIN:VEVENT`, `0` in the split list is the header of the calendar file, and the ret are all of the individual events, which I need to add `BEGIN:VEVENT` back in for later. 

In [ ]:
events_raw = calendar_txt.split("BEGIN:VEVENT")
ical_header = events_raw[0]
events = ["BEGIN:VEVENT" + x.replace("END:VCALENDAR","") for x in events_raw[1:]]
for event in events:
    print("#"*80)
    print(event)

Microsoft FindTime gave me a note that 20 is the most number of options to give, so I went with giving each group a quarter of the results (line 19).

I also made a separate dictionary to count the number of times I gave out an event, and then made a function to list all of the events that were used the max number of times. 

Then at the end I initialize the events available for a group as an empty list, create a list of items that are already in the list or have been used more than 2 times (line 21). After that, `random.choice` a new event and add it to the current groups list. 

In [ ]:
import math

event_usage_dict = {}

for event in events:
    event_usage_dict[event] = 0

date_dict = {}

def morethan(counterdict, maxv=3):
    exception_list = []
    for entry in counterdict.keys():
        if counterdict[entry] >= maxv:
            exception_list.append(entry)
    return(exception_list)    

for group in phd_groups:
    date_dict[group] = []
    n = math.ceil(len(events)/4)
    for i in range(n):
        negate_list = date_dict[group] + morethan(event_usage_dict, 2)
        available_event_list = [x for x in events if x not in negate_list]
        choice = rc(available_event_list)
        event_usage_dict[choice] += 1
        date_dict[group].append(choice)
    print(len(date_dict[group]))
    
print(date_dict)

In [ ]:
print(event_usage_dict)

In [ ]:
print(ical_header)

Here I just write out the different group lists to a file, replacing the name of the original calendar's heading with the current group. 

In [ ]:
import os
odir = r"C:\Users\SPEED001\Documents\GitHub\random"
for group in phd_groups:
    ofname = odir + os.sep + group + ".ics"
    new_cal_text = ""
    new_cal_text += ical_header.replace("PhD Training Time Slots", f"{group} Time Slots")
    for event in date_dict[group]:
        new_cal_text += event
    new_cal_text += "END:VCALENDAR"
    #print(new_cal_text)
    with open(ofname, "w") as newcal:
        newcal.write(new_cal_text)